# SQL Practice Notebook
---

### Introduction:

"`SQL` (**S**tructured **Q**uery **L**anguage) is a programming language designed for managing data in a relational [database](https://en.wikipedia.org/wiki/Database). It's been around since the 1970s and is the most common method of accessing data in databases today. SQL has a variety of functions that allow its users to read, manipulate, and change data. Though SQL is commonly used by engineers in software development, it's also popular with data analysts for a few reasons:

- It's **semantically easy** to understand and learn.
- Because it can be used to **access large amounts of data directly where it's stored**, analysts don't have to copy data into other applications.
- Compared to spreadsheet tools, data analysis done in **SQL is easy to audit and replicate**. For analysts, this means no more looking for the cell with the typo in the formula.

SQL is great for performing the types of aggregations that you might normally do in an `Excel pivot table—sums`, `ccounts`, `minimums and maximums`, etc.—but over much **larger datasets and on multiple tables at the same time**." [[8]](https://mode.com/sql-tutorial/introduction-to-sql/)

### Objectives:

 The main objective of this notebook is practicing basic SQL querying using sqlite3 python library. 
    
### Dataset:
    
I will be using the World Development Indicator (WDI) dataset and we'll be querying about a subset of world countries; I chose the Sub-Saharan Africa region.
    
### Inspiration:

This notebook is hugely inspired by [the notebook (ref.1)](https://www.kaggle.com/code/jonaspalucibarbosa/wdi-brazil-unemployment-using-sql-for-analysis) of Jonas Paluci Barbosa who made a similar notebook using the same dataset. 




In [1]:
import sqlite3
import numpy as np 
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
PATH_TO_SQLITE_MASTER = '../input/world-development-indicators/database.sqlite'

## 1. Explore the database
### 1.1 List all the tables in the database.

We see from the right hand side of the notebook all the tables in the sqlite database and this step may not be necessary. Nevertheless, as this is a practice notebook we will list them anyway. We will note that there are 6 tables. 

In [3]:
## (ref.2) https://www.geeksforgeeks.org/how-to-list-tables-using-sqlite3-in-python/
try:    
    connect = sqlite3.connect(PATH_TO_SQLITE_MASTER)
    print("Connected to SQLite")
    
    sql_query = """
    SELECT name 
    FROM sqlite_master
    WHERE type='table';
    """
    cursor = connect.cursor()
    cursor.execute(sql_query)
       
    print("List of tables\n")
    print(cursor.fetchall())
    
    
except sqlite3.Error as error:
    
    print("Failed to execute the above query", error)
        

Failed to execute the above query unable to open database file


Alternatively we can also print a dataframe of the tabels containg all information as shown below.

In [ ]:
df_tables =  pd.read_sql("""
    SELECT * 
    FROM sqlite_master
    WHERE type='table';
    """,
            con = connect            
)
df_tables



### 1.2 The tables
#### 1.2.1 Country

Column-wise this is the biggest table with 31 columns and has entries for 247 countries. This table contains several information about the data collected for all countries. 

In [ ]:
df_country =  pd.read_sql("""

SELECT *
FROM Country
""",
            con = connect            
)
display(df_country.head(3))
df_country.shape

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
1,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006
2,DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,...,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001


(247, 31)

In [ ]:
df_sub_saharan_africa = df_country[df_country['Region'] == 'Sub-Saharan Africa']
SSA = df_sub_saharan_africa.ShortName.tolist()
display('The Sub-Saharan Africa has {} countries' .format(len(SSA)))

SSA[::5]

'The Sub-Saharan Africa has 48 countries'

['Angola',
 'Cabo Verde',
 'Congo',
 'Ethiopia',
 'Kenya',
 'Mali',
 'Niger',
 'Seychelles',
 'Sudan',
 'Uganda']

#### 1.2.2 CountryNotes

This table has 4857 rows and 3 columns; Countrycode, Seriescode and Descriptions. 

In [ ]:
df_country_notes =  pd.read_sql("""

SELECT *
FROM CountryNotes 
""",
            con = connect            
)
df_country_notes.head(3)

,Countrycode,Seriescode,Description
0,ABW,EG.EGY.PRIM.PP.KD,Sources: Estimated based on UN Energy Statisti...
1,ABW,EG.ELC.RNEW.ZS,Sources: UN Energy Statistics (2014)
2,ABW,EG.FEC.RNEW.ZS,Sources: UN Energy Statistics (2014)


#### 1.2.3 Footnotes

This table contains 532415 rows and 4 columns (Countrycode, Seriescode, Year and Description). 

In [ ]:
df_footnotes =  pd.read_sql(""" 
SELECT * 
FROM Footnotes 
""", 
                            con = connect)
df_footnotes.tail(3)

,Countrycode,Seriescode,Year,Description
532412,ZWE,TX.VAL.MRCH.XD.WD,YR2008,Source: UNCTAD Handbook of Statistics and data...
532413,ZWE,TX.VAL.MRCH.XD.WD,YR2009,Source: UNCTAD Handbook of Statistics and data...
532414,ZWE,TX.VAL.MRCH.XD.WD,YR2010,Source: UNCTAD Handbook of Statistics and data...


#### 1.2.4. Indicators

Row-wise this is the largest table with 5656458 rows and 6 columns. This is in a way the `"data table"` of the WDI dataset. It contains the reported values (measured/modelled) of all indicators for all countries in the countries-list for the years 1960 to 2014.

In [ ]:
df_indicators =  pd.read_sql("""

SELECT *
FROM Indicators
""",
            con = connect            
)
df_indicators.head(3)

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,133.560907
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,87.797601
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579


In [ ]:
df_last =  pd.read_sql("""
SELECT MAX(Year) AS LatestYear
FROM Indicators
""",
            con = connect            
)

df_start =  pd.read_sql("""
SELECT MIN(Year) AS EarliestYear
FROM Indicators
""",
            con = connect            
)
pd.concat([df_start, df_last], axis=1)

,EarliestYear,LatestYear
0,1960,2015


#### 1.2.5. Series

This table has 1345 rows and 20 columns. 

In [ ]:
df_series =  pd.read_sql("""

SELECT *
FROM Series
""",
            con = connect            
)
df_series.head(3)

,SeriesCode,Topic,IndicatorName,ShortDefinition,LongDefinition,UnitOfMeasure,Periodicity,BasePeriod,OtherNotes,AggregationMethod,LimitationsAndExceptions,NotesFromOriginalSource,GeneralComments,Source,StatisticalConceptAndMethodology,DevelopmentRelevance,RelatedSourceLinks,OtherWebLinks,RelatedIndicators,LicenseType
0,BN.KLT.DINV.CD,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net (BoP, current US$)",,Foreign direct investment are the net inflows ...,,Annual,,,,,,Note: Data are based on the sixth edition of t...,"International Monetary Fund, Balance of Paymen...",,,,,,Open
1,BX.KLT.DINV.WD.GD.ZS,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net inflows (% of GDP)",,Foreign direct investment are the net inflows ...,,Annual,,,Weighted average,FDI data do not give a complete picture of int...,,Note: Data starting from 2005 are based on the...,"International Monetary Fund, International Fin...",Data on equity flows are based on balance of p...,Private financial flows - equity and debt - ac...,,,,Open
2,BX.KLT.DINV.CD.WD,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net inflows (BoP, c...",,Foreign direct investment refers to direct inv...,,Annual,,,Sum,FDI data do not give a complete picture of int...,,Note: Data starting from 2005 are based on the...,"International Monetary Fund, Balance of Paymen...",Data on equity flows are based on balance of p...,Private financial flows - equity and debt - ac...,,,,Open


#### 1.2.6. SeriesNotes

This table has 369 rows and 3 columns, i.e `Seriescode`, `Year`, and `Description`. The table is all about the notes oe descriptions (as given in the `description` column) of each Seriescode for the corresponding year. For example, we can infer from the table that for year 1960 the data for `Health: Reproductive health (Seciecode SP.ADO.TFRT)` was calculated by interpolating using data for 1957 and 1962. 

In [ ]:
df_seriesnotes =  pd.read_sql("""

SELECT *
FROM SeriesNotes
""",
            con = connect            
)
df_seriesnotes.head(3)

,Seriescode,Year,Description
0,SP.ADO.TFRT,YR1960,Interpolated using data for 1957 and 1962.
1,SP.DYN.AMRT.FE,YR1960,"Interpolated using data for 1957 and 1962, if ..."
2,SP.DYN.AMRT.MA,YR1960,"Interpolated using data for 1957 and 1962, if ..."


### 2. Query and Analyse

We have now seen that what the tables are and the details of their columns we can query and do futher analysis on a selected topics. Sub-Saharan Africa region is selected as our area of interest and we have selected few questions to ponder. We will limit our focus to topics such as economy,health, natural resources, education and so on. Example question would look like:

1. Which countries have the most economic growth (measured in GDP) in the last 5 years?
2. What is the contribution of natural resources to the countries GDP?
3. What is the health expenditure in each country? And their per capita health expenditure?
4. Education, Child Labor, etc..


In [ ]:
df_SSA = pd.read_sql("""

SELECT *
FROM Country
WHERE Region = 'Sub-Saharan Africa'
 
""",
            con = connect            
)

print('Number of countries in the Sub-Saharan Africa region is {}'.format(df_SSA.shape[0]))
print('\n')
df_SSA.head(3)

Number of countries in the Sub-Saharan Africa region is 48




,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,"April 2013 database update: Based on IMF data,...",Sub-Saharan Africa,Upper middle income,AO,...,Budgetary central government,General Data Dissemination System (GDDS),2014,"Malaria Indicator Survey (MIS), 2011","Integrated household survey (IHS), 2008/09",,2015,,,2005
1,BEN,Benin,Benin,Republic of Benin,BJ,West African CFA franc,,Sub-Saharan Africa,Low income,BJ,...,Budgetary central government,General Data Dissemination System (GDDS),2013,"Multiple Indicator Cluster Survey (MICS), 2014",Core Welfare Indicator Questionnaire Survey (C...,,2011/12,,2013,2001
2,BWA,Botswana,Botswana,Republic of Botswana,BW,Botswana pula,Fiscal year end: March 31; reporting period fo...,Sub-Saharan Africa,Upper middle income,BW,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Multiple Indicator Cluster Survey (MICS), 2000","Expenditure survey/budget survey (ES/BS), 2009/10",,2011. Population and Housing Census.,2011,2013,2000


### Topics Details

The WDI has several topics covered. These are within the Series table. Let's now print out the list of topics so that we can select the topics of our interest for further analysis. 

In [ ]:
topics = pd.read_sql("""
SELECT DISTINCT Topic
FROM Series;
""",
                     connect)

topics.Topic.tolist()

['Economic Policy & Debt: Balance of payments: Capital & financial account',
 'Economic Policy & Debt: Balance of payments: Current account: Balances',
 'Economic Policy & Debt: Balance of payments: Current account: Goods, services & income',
 'Economic Policy & Debt: Balance of payments: Current account: Transfers',
 'Economic Policy & Debt: Balance of payments: Reserves & other items',
 'Economic Policy & Debt: External debt: Amortization',
 'Economic Policy & Debt: External debt: Arrears, reschedulings, etc.',
 'Economic Policy & Debt: External debt: Commitments',
 'Economic Policy & Debt: External debt: Currency composition',
 'Economic Policy & Debt: External debt: Debt outstanding',
 'Economic Policy & Debt: External debt: Debt ratios & other items',
 'Economic Policy & Debt: External debt: Debt service',
 'Economic Policy & Debt: External debt: Disbursements',
 'Economic Policy & Debt: External debt: Interest',
 'Economic Policy & Debt: External debt: Net flows',
 'Economic Poli

### 2.1 GDP Growth (%)

"Gross domestic product (GDP) is the total monetary or market value of all the finished goods and services produced within a country’s borders in a specific time period. As a broad measure of overall domestic production, it functions as a comprehensive scorecard of a given country’s economic health.

"The GDP growth rate compares the year-over-year (or quarterly) change in a country’s economic output to measure how fast an economy is growing. Usually expressed as a percentage rate, this measure is popular for economic policy-makers because GDP growth is thought to be closely connected to key policy targets such as inflation and unemployment rates." [[4](https://www.investopedia.com/terms/g/gdp.asp/)]

Now let's find the right tables and extract the GDP growth for the sub-sharah Africa for the years 2010 - 2014 (last five years according to the dataset).

> Now let's query the `Series` table with a key work `Growth rates` and then selelect the right `SeriesCode` for the GDP growth(annual %)

> The SerierCode we are interested in is `NY.GDP.MKTP.KD.ZG`. We can use this code to query the Indicators table.


In [ ]:
indicator_type_growthRates = pd.read_sql("""
SELECT SeriesCode, IndicatorName, LongDefinition
FROM Series
WHERE Topic LIKE '%Growth rates%';
""", 
                             connect)
indicator_type_growthRates

,SeriesCode,IndicatorName,LongDefinition
0,NV.AGR.TOTL.KD.ZG,"Agriculture, value added (annual % growth)",Annual growth rate for agricultural value adde...
1,NE.EXP.GNFS.KD.ZG,Exports of goods and services (annual % growth),Annual growth rate of exports of goods and ser...
2,NE.CON.TETC.KD.ZG,"Final consumption expenditure, etc. (annual % ...",Average annual growth of final consumption exp...
3,NY.GDP.MKTP.KD.ZG,GDP growth (annual %),Annual percentage growth rate of GDP at market...
4,NY.GDP.PCAP.KD.ZG,GDP per capita growth (annual %),Annual percentage growth rate of GDP per capit...
5,NE.CON.GOVT.KD.ZG,General government final consumption expenditu...,Annual percentage growth of general government...
6,NE.GDI.TOTL.KD.ZG,Gross capital formation (annual % growth),Annual growth rate of gross capital formation ...
7,NE.GDI.FTOT.KD.ZG,Gross fixed capital formation (annual % growth),Average annual growth of gross fixed capital f...
8,NE.CON.PRVT.KD.ZG,Household final consumption expenditure (annua...,Annual percentage growth of household final co...
9,NE.CON.PRVT.PC.KD.ZG,Household final consumption expenditure per ca...,Annual percentage growth of household final co...


In [ ]:
df_SSA_GDP = pd.read_sql("""

SELECT Year, Value AS GDP, CountryName
FROM Indicators
WHERE CountryName IN 

('Angola',
 'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
 'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
 'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
 'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
 'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
 'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
 'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
 'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND  
 
 IndicatorCode = 'NY.GDP.MKTP.KD.ZG' AND Year IN (2010, 2011, 2012, 2013, 2014) 

 """, connect)

df_SSA_GDP.head(3)

,Year,GDP,CountryName
0,2010,2.116850,Benin
1,2010,8.563632,Botswana
2,2010,8.446325,Burkina Faso


#### Remark:
> We know that we have 48 countries for the Sub-Saharan Africa region. However, for various reasons, each contry many not have the data we are interested in. Let's write a function which checks how many of the 48 countries are in the specific query.

In [ ]:
def CountriesWithNoData(df):    
    print('Only {} countries out of the {} have data for our current query.'.format(len(df.CountryName.unique()), len(df_SSA)))
    print('')
    CoutriesWithNoData = [country for country in SSA if country not in df.CountryName.unique()]
    print('Countries with no data in our current query are: ')
    print('')
    for i, country in enumerate(CoutriesWithNoData):
        print(i+1, country)

In [ ]:
CountriesWithNoData(df_SSA_GDP)

Only 41 countries out of the 48 have data for our current query.

Countries with no data in our current query are: 

1 Angola
2 Congo
3 Côte d'Ivoire
4 Dem. Rep. Congo
5 São Tomé and Principe
6 Somalia
7 The Gambia


Now let's visualize the data. We will be using a plotly bar plot. What we see below is the average GDP growth rate of the countries for the years 2010 to 2014. The yearly breakdown is given in the heatmap that follows. 

In [ ]:
df_gdp = df_SSA_GDP.groupby(["CountryName"])['GDP'].mean().sort_values(ascending=True).reset_index()

fig = go.Figure
fig = go.Figure(data=[go.Bar(y=df_gdp['CountryName'], 
                             x=df_gdp['GDP'],
                             orientation='h'
                            ),                     
                     ]
               )

fig.update_layout(title='<b>Annual GDP Growth (%): 2010-2014<b>',
                  yaxis_title='<b>Countries<b>',
                  xaxis_title='<b> GDP Growth (%) <b>',
                  titlefont={'size': 20, 'family':'Serif'},
                  template='simple_white',
                  width=750, height=800,
                )

colors = ['lightseagreen',] * len(df_gdp)
colors[0] = 'crimson' 
colors[1] = 'crimson' 
colors[2] = 'crimson'



fig.update_traces(marker_color=colors, marker_line_color=None,
                  marker_line_width=2.5, opacity=None)
fig.show()

In [ ]:
_ = df_SSA_GDP.pivot(index = 'Year', columns = 'CountryName', values='GDP')

display(_.T.head(10))

fig = px.imshow(_, color_continuous_scale='earth', origin='lower')
fig.update_layout(title='<b> GDP Growth Sub-Saharan Africa: 2010- 2014',
                 template='simple_white',
                 width=850, height=400)
fig.show()

Year,2010,2011,2012,2013,2014
CountryName,,,,,
Benin,2.116850,2.962750,4.643023,6.878996,6.541487
Botswana,8.563632,6.048316,4.831543,9.322202,4.421041
Burkina Faso,8.446325,6.521769,6.452675,3.649305,3.996045
Burundi,3.785903,4.191626,4.019366,4.594054,4.660918
Cabo Verde,1.466913,3.968784,1.081918,1.045244,2.799847
Cameroon,3.268649,4.140592,4.588739,5.561688,5.926965
Central African Republic,3.046657,3.300740,4.112756,-36.047079,1.010329
Chad,13.550101,0.082870,8.882576,5.700001,7.300001
Comoros,2.200000,2.600000,3.000000,3.500000,2.061639


#### Observations: 
- Ethiopia has the highest overall GDP growth in the years 2010- 2015
- Siera Leone resgisters the higest GDP growth of 20.9% in the year 2013.
- South Sudan (2012) and Central African Republic (2013) had the worst GDP loss of 46% and 36% respectively.
- Eritrea's has not reported GDP values from 2012 - 2014 for some reason.

### 2.2 Natural resources
#### Total natural resources rents (% of GDP)

"This indicator measures total natural resources rents as a share of the gross domestic product (GDP) of a given country. Total natural resources rents are the sum of `oil rents`, `natural gas rents`, `coal rents (hard and soft)`, `mineral rents`, and `forest rents`. Accounting for the contribution of natural resources to economic output is important in building an analytical framework for sustainable development. In some countries earnings from natural resources, especially from fossil fuels and minerals, account for a sizable share of GDP, and much of these earnings come in the form of economic rents - revenues above the cost of extracting the resources. Natural resources give rise to economic rents because they are not produced. For produced goods and services competitive forces expand supply until economic profits are driven to zero, but natural resources in fixed supply often command returns well in excess of their cost of production. Rents from nonrenewable resources - fossil fuels and minerals - as well as rents from overharvesting of forests indicate the liquidation of a country's capital stock. When countries use such rents to support current consumption rather than to invest in new capital to replace what is being used up, they are, in effect, borrowing against their future "[[5](https://landportal.org/taxonomy/term/8165)]

> Similar to the previous query, let's query the `Series` table with a key work `Natural resources contribution to GDP` and then selelect the right `SeriesCode` .

> The SerierCode we are interested in is `NY.GDP.TOTL.RT.ZS`. We can use this code to query the Indicators table.


In [ ]:
indicator_type_naturalResources = pd.read_sql("""
SELECT SeriesCode, IndicatorName, LongDefinition
FROM Series
WHERE Topic LIKE '%Natural resources contribution to GDP%';
""", 
                             connect)
indicator_type_naturalResources

,SeriesCode,IndicatorName,LongDefinition
0,NY.GDP.COAL.RT.ZS,Coal rents (% of GDP),Coal rents are the difference between the valu...
1,NY.GDP.FRST.RT.ZS,Forest rents (% of GDP),Forest rents are roundwood harvest times the p...
2,NY.GDP.MINR.RT.ZS,Mineral rents (% of GDP),Mineral rents are the difference between the v...
3,NY.GDP.NGAS.RT.ZS,Natural gas rents (% of GDP),Natural gas rents are the difference between t...
4,NY.GDP.PETR.RT.ZS,Oil rents (% of GDP),Oil rents are the difference between the value...
5,NY.GDP.TOTL.RT.ZS,Total natural resources rents (% of GDP),Total natural resources rents are the sum of o...


In [ ]:
for text in enumerate(indicator_type_naturalResources['LongDefinition'].tolist()):
    print(text)
    print('')

(0, 'Coal rents are the difference between the value of both hard and soft coal production at world prices and their total costs of production.')

(1, 'Forest rents are roundwood harvest times the product of average prices and a region-specific rental rate.')

(2, 'Mineral rents are the difference between the value of production for a stock of minerals at world prices and their total costs of production. Minerals included in the calculation are tin, gold, lead, zinc, iron, copper, nickel, silver, bauxite, and phosphate.')

(3, 'Natural gas rents are the difference between the value of natural gas production at world prices and total costs of production.')

(4, 'Oil rents are the difference between the value of crude oil production at world prices and total costs of production.')

(5, 'Total natural resources rents are the sum of oil rents, natural gas rents, coal rents (hard and soft), mineral rents, and forest rents.')



In [ ]:
df_SSA_NatRes = pd.read_sql("""

SELECT Year, Value AS NaturalResource, CountryName
FROM Indicators
WHERE CountryName IN 

('Angola',
 'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
 'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
 'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
 'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
 'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
 'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
 'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
 'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND IndicatorCode = 'NY.GDP.TOTL.RT.ZS' AND Year IN (2010, 2011, 2012, 2013, 2014)
  
 """, connect)

df_SSA_NatRes.head(3)

,Year,NaturalResource,CountryName
0,2010,45.992915,Angola
1,2010,5.068527,Benin
2,2010,5.538524,Botswana


In [ ]:
CountriesWithNoData(df_SSA_NatRes)

Only 42 countries out of the 48 have data for our current query.

Countries with no data in our current query are: 

1 Congo
2 Côte d'Ivoire
3 Dem. Rep. Congo
4 São Tomé and Principe
5 Somalia
6 The Gambia


In [ ]:
x = df_SSA_NatRes.groupby(["CountryName"])['NaturalResource'].mean().index
y = df_SSA_NatRes.groupby(["CountryName"])['NaturalResource'].mean()
fig = go.Figure
fig = go.Figure(data=[go.Bar(x=x, y=y, marker_color='lightseagreen')]).update_xaxes(categoryorder='total descending')

fig.update_layout(title='<b>Total Natural Resources (% GDP) <b>',
                  xaxis_title='<b>Countries<b>',
                  yaxis_title='<b>% GDP<b>',
                  titlefont={'size': 20, 'family':'Serif'},
                  template='simple_white',
                  width=850, height=550,                  
                )

fig.show()
              

#### Observations:
- Equatorial Guinea's is the highest in the list with around 63% of its GDP coming from natural resources, followed by Gabon and Mauritania.
- Zooming to the other end of the chart, Mauritius and the other two islands have little to no GDP share coming from the 'natural resources'. 

"Mauritius is an African island nation that occupies an area of roughly 790 square miles that is situated within the Indian Ocean. According to the World Bank, the economy of Mauritius had grown significantly since gaining independence in 1968 when it was considered a low-income economy. One of the leading economic experts at the time, Professor James Meade, believed that the nation had very little chance of making significant economic progress. He thought that due to Mauritius' small size, it would be forever trapped in a Malthusian trap which would limit its economic potential. Despite the bleak picture that Professor Meade painted, Mauritius was able to fully utilize the natural resources available to achieve substantial economic growth. `Some of the nation's natural resources include the arable land, the beautiful scenery, and the fish`." [[Ref.7]](https://www.worldatlas.com/articles/what-are-the-major-natural-resources-of-mauritius.html)

### 2.3 Health

The data contains many health related topics. For our analysis we have selected only two topics, the countries `total health expenditure`, and the `per capita health expenditures`.

Term defination as per the WDI tables. 

**Total health expenditure**: Total health expenditure is the sum of public and private health expenditure. It covers the provision of health services (preventive and curative), family planning activities, nutrition activities, and emergency aid designated for health but does not include provision of water and sanitation.

**Health expenditure per capita (current US$)**: Health expenditure per capita is the sum of public and private health expenditures as a ratio of total population. It covers the provision of health services (preventive and curative), family planning activities, nutrition activities, and emergency aid designated for health but does not include provision of water and sanitation. Data are in current U.S. dollars.

> Now let's query the `Series` table with a key work `Health sevice`.

In [ ]:
indicator_type_health = pd.read_sql("""
SELECT SeriesCode, IndicatorName, LongDefinition
FROM Series
WHERE Topic LIKE '%Health services';
""", 
                             connect)
indicator_type_health

,SeriesCode,IndicatorName,LongDefinition
0,SH.MED.CMHW.P3,"Community health workers (per 1,000 people)",Community health workers include various types...
1,SH.XPD.EXTR.ZS,External resources for health (% of total expe...,External resources for health are funds or ser...
2,SH.XPD.PCAP,Health expenditure per capita (current US$),Total health expenditure is the sum of public ...
3,SH.XPD.PCAP.PP.KD,"Health expenditure per capita, PPP (constant 2...",Total health expenditure is the sum of public ...
4,SH.XPD.PRIV.ZS,"Health expenditure, private (% of GDP)",Private health expenditure includes direct hou...
5,SH.XPD.PUBL.ZS,"Health expenditure, public (% of GDP)",Public health expenditure consists of recurren...
6,SH.XPD.PUBL.GX.ZS,"Health expenditure, public (% of government ex...",Public health expenditure consists of recurren...
7,SH.XPD.PUBL,"Health expenditure, public (% of total health ...",Public health expenditure consists of recurren...
8,SH.XPD.TOTL.ZS,"Health expenditure, total (% of GDP)",Total health expenditure is the sum of public ...
9,SH.MED.BEDS.ZS,"Hospital beds (per 1,000 people)",Hospital beds include inpatient beds available...


In [ ]:
for text in enumerate(indicator_type_health['IndicatorName'].tolist()):
    print(text)
    print('')

(0, 'Community health workers (per 1,000 people)')

(1, 'External resources for health (% of total expenditure on health)')

(2, 'Health expenditure per capita (current US$)')

(3, 'Health expenditure per capita, PPP (constant 2011 international $)')

(4, 'Health expenditure, private (% of GDP)')

(5, 'Health expenditure, public (% of GDP)')

(6, 'Health expenditure, public (% of government expenditure)')

(7, 'Health expenditure, public (% of total health expenditure)')

(8, 'Health expenditure, total (% of GDP)')

(9, 'Hospital beds (per 1,000 people)')

(10, 'Nurses and midwives (per 1,000 people)')

(11, 'Out-of-pocket health expenditure (% of private expenditure on health)')

(12, 'Out-of-pocket health expenditure (% of total expenditure on health)')

(13, 'Physicians (per 1,000 people)')



**The two IndicatorCode we are interested in are:**

> `SH.XPD.TOTL.ZS` : Health expenditure, total (% of GDP)

> `SH.XPD.PCAP` : Health expenditure per capita (current US$)

In [ ]:
df_SSA_THE_0 = pd.read_sql("""

SELECT Year, Value AS Total_Health_Expenditure, CountryName
FROM Indicators
WHERE CountryName IN 

('Angola',
 'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
 'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
 'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
 'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
 'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
 'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
 'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
 'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND  
 
 IndicatorCode = 'SH.XPD.TOTL.ZS' AND Year IN (2010, 2011, 2012, 2013, 2014)
 
 """, connect)

df_SSA_THE_1 = pd.read_sql("""

SELECT Year, Value AS Health_Expenditure_perCapita, CountryName
FROM Indicators
WHERE CountryName IN 

('Angola',
 'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
 'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
 'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
 'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
 'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
 'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
 'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
 'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND  
 
 IndicatorCode = 'SH.XPD.PCAP' AND Year IN (2010, 2011, 2012, 2013, 2014)
 
 """, connect)

In [ ]:
# (ref.3) https://stackoverflow.com/questions/44545921/in-pandas-how-to-concatenate-horizontally-and-then-remove-the-redundant-columns

df_SSA_THE = pd.concat([df_SSA_THE_0, df_SSA_THE_1], axis=1).T.drop_duplicates().T
df_SSA_THE.head(3)

,Year,Total_Health_Expenditure,CountryName,Health_Expenditure_perCapita
0,2010,3.408258,Angola,143.782461
1,2010,4.689867,Benin,32.343589
2,2010,5.642684,Botswana,393.879741


In [ ]:
CountriesWithNoData(df_SSA_THE)

Only 41 countries out of the 48 have data for our current query.

Countries with no data in our current query are: 

1 Congo
2 Côte d'Ivoire
3 Dem. Rep. Congo
4 São Tomé and Principe
5 Somalia
6 The Gambia
7 Zimbabwe


In [ ]:
x = df_SSA_THE.groupby(["CountryName"])['Total_Health_Expenditure'].mean().index
y = df_SSA_THE.groupby(["CountryName"])['Total_Health_Expenditure'].mean()

fig = go.Figure
fig = go.Figure(data=[go.Bar(x=x, y=y, marker_color='lightseagreen'),
                      
                     ]
               ).update_xaxes(categoryorder='total descending')

fig.update_layout(title='<b>Total Health Expenditure (% GDP) <b>',
                  xaxis_title='<b>Countries<b>',
                  yaxis_title='<b>Expenditure (% GDP) <b>',
                  titlefont={'size': 20, 'family':'Serif'},
                  template='simple_white',
                  width=850, height=550,
                )

fig.show()

In [ ]:
df_h = df_SSA_THE.groupby(["CountryName"])['Health_Expenditure_perCapita'].mean().sort_values(ascending=False).reset_index()

fig = go.Figure
fig = go.Figure(data=[go.Bar(x=df_h['CountryName'] , 
                             y=df_h["Health_Expenditure_perCapita"], 
                             marker_color='crimson'),                      
                     ]
               )

fig.update_layout(title='<b>Health Expenditure Per Capita<b>',
                  xaxis_title='<b>Countries<b>',
                  yaxis_title='<b>Expenditure per Capita (USD) <b>',
                  titlefont={'size': 20, 'family':'Serif'},
                  template='simple_white',
                  width=850, height=550,
                )
fig.show()

#### Observations: 
- Lesotho, Sierra Leone, Rwanda, Liberia and Uganda are the two five countries that have the highest health expenditure in the SSA region. Around 10-11% of their GDP is dedicated to health expenditure. At 2% South Sudan is at the bottom of the list. 
- However, looking at the per capita expenditure, Equatorial Guinea and South Africa are at the top of the pile with 710 and 632 USD respectively. Eritrea is at the bottom of the list with 14 USD health expenditure per person.
- About half of the countries spend less than 50 USD per capita on health!


### 2.4 Education

"Of all regions, `sub-Saharan Africa` has the highest rates of **education exclusion**. Over one-fifth of children between the ages of about 6 and 11 are out of school, followed by one-third of youth between the ages of about 12 and 14. According to UIS data, almost 60% of youth between the ages of about 15 and 17 are not in school. 

"Without urgent action, the situation will likely get worse as the region faces a rising demand for education due to a still-growing school-age population.

"Girls’ education is a major priority. Across the region, 9 million girls between the ages of about 6 and 11 will never go to school at all, compared to 6 million boys, according to UIS data. Their disadvantage starts early: 23% of girls are out of primary school compared to 19% of boys. By the time they become adolescents, the exclusion rate for girls is 36% compared to 32% for boys." [[Ref.]](http://uis.unesco.org/en/topic/education-africa)

> Now let's see what the data says regarding out of school children. We are interested in the boys and girls data.

> For the boys the indicatorCode is `SE.PRM.UNER.MA` and for the girls it is `SE.PRM.UNER.FE`

In [ ]:
indicator_type_education = pd.read_sql("""
SELECT SeriesCode, IndicatorName, LongDefinition
FROM Series
WHERE Topic LIKE '%Education: Participation';
""", 
                             connect)
indicator_type_education

,SeriesCode,IndicatorName,LongDefinition
0,SE.PRM.TENR,"Adjusted net enrollment rate, primary (% of pr...",Adjusted net enrollment is the number of pupil...
1,SE.PRM.TENR.FE,"Adjusted net enrollment rate, primary, female ...",Adjusted net enrollment is the number of pupil...
2,SE.PRM.TENR.MA,"Adjusted net enrollment rate, primary, male (%...",Adjusted net enrollment is the number of pupil...
3,SE.PRM.UNER,"Children out of school, primary",Children out of school are the number of prima...
4,SE.PRM.UNER.FE,"Children out of school, primary, female",Children out of school are the number of prima...
5,SE.PRM.UNER.MA,"Children out of school, primary, male",Children out of school are the number of prima...
6,SE.PRM.ENRL,"Primary education, pupils",Primary education pupils is the total number o...
7,SE.PRM.ENRL.FE.ZS,"Primary education, pupils (% female)",Female pupils as a percentage of total pupils ...
8,SE.PRE.ENRR,"School enrollment, preprimary (% gross)",Gross enrollment ratio is the ratio of total e...
9,SE.PRE.ENRR.FE,"School enrollment, preprimary, female (% gross)",Gross enrollment ratio is the ratio of total e...


In [ ]:
for text in enumerate(indicator_type_education['IndicatorName'].tolist()):
    print(text)
    print('')

(0, 'Adjusted net enrollment rate, primary (% of primary school age children)')

(1, 'Adjusted net enrollment rate, primary, female (% of primary school age children)')

(2, 'Adjusted net enrollment rate, primary, male (% of primary school age children)')

(3, 'Children out of school, primary')

(4, 'Children out of school, primary, female')

(5, 'Children out of school, primary, male')

(6, 'Primary education, pupils')

(7, 'Primary education, pupils (% female)')

(8, 'School enrollment, preprimary (% gross)')

(9, 'School enrollment, preprimary, female (% gross)')

(10, 'School enrollment, preprimary, male (% gross)')

(11, 'School enrollment, primary (% gross)')

(12, 'School enrollment, primary (% net)')

(13, 'School enrollment, primary (gross), gender parity index (GPI)')

(14, 'School enrollment, primary and secondary (gross), gender parity index (GPI)')

(15, 'School enrollment, primary, female (% gross)')

(16, 'School enrollment, primary, female (% net)')

(17, 'School enroll

In [ ]:
df_SSA_EDUC_OutSchool_boys = pd.read_sql("""

SELECT Year, Value AS Child_OutOfSchool_Prim_boys, CountryName
FROM Indicators
WHERE CountryName IN 

('Angola',
 'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
 'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
 'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
 'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
 'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
 'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
 'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
 'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND  
 
 IndicatorCode = 'SE.PRM.UNER.MA' AND Year IN (2010, 2011, 2012, 2013, 2014)
  
 """, connect)

df_SSA_EDUC_OutSchool_boys.head(3)

,Year,Child_OutOfSchool_Prim_boys,CountryName
0,2010,119896,Angola
1,2010,492409,Burkina Faso
2,2010,39887,Burundi


In [ ]:
CountriesWithNoData(df_SSA_EDUC_OutSchool_boys)

Only 32 countries out of the 48 have data for our current query.

Countries with no data in our current query are: 

1 Congo
2 Côte d'Ivoire
3 Dem. Rep. Congo
4 Eritrea
5 Ethiopia
6 Gabon
7 Madagascar
8 Malawi
9 São Tomé and Principe
10 Seychelles
11 Sierra Leone
12 Somalia
13 South Africa
14 Swaziland
15 The Gambia
16 Togo


In [ ]:
df_SSA_EDUC_OutOfSchool_girls = pd.read_sql("""

SELECT Year, 
    Value AS Child_OutOfSchool_Prim_girls, 
    CountryName
FROM Indicators
WHERE 
    CountryName IN 
        ('Angola',
         'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
         'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
         'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
         'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
         'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
         'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
         'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
         'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND
     IndicatorCode = 'SE.PRM.UNER.FE' 
 AND 
     Year IN (2010, 2011, 2012, 2013, 2014)
 
 """, connect)

df_SSA_EDUC_OutOfSchool_girls.head(3)

,Year,Child_OutOfSchool_Prim_girls,CountryName
0,2010,376262,Angola
1,2010,540522,Burkina Faso
2,2010,41263,Burundi


In [ ]:
CountriesWithNoData(df_SSA_EDUC_OutOfSchool_girls)

Only 32 countries out of the 48 have data for our current query.

Countries with no data in our current query are: 

1 Congo
2 Côte d'Ivoire
3 Dem. Rep. Congo
4 Eritrea
5 Ethiopia
6 Gabon
7 Madagascar
8 Malawi
9 São Tomé and Principe
10 Seychelles
11 Sierra Leone
12 Somalia
13 South Africa
14 Swaziland
15 The Gambia
16 Togo


In [ ]:
OutOfSchool = pd.concat([df_SSA_EDUC_OutOfSchool_girls, df_SSA_EDUC_OutSchool_boys], axis=1).T.drop_duplicates().T

In [ ]:
xg = OutOfSchool.groupby(["CountryName"])['Child_OutOfSchool_Prim_girls'].mean().index
yg = OutOfSchool.groupby(["CountryName"])['Child_OutOfSchool_Prim_girls'].mean()

xb = OutOfSchool.groupby(["CountryName"])['Child_OutOfSchool_Prim_boys'].mean().index
yb = OutOfSchool.groupby(["CountryName"])['Child_OutOfSchool_Prim_boys'].mean()

fig = go.Figure
fig = go.Figure(data=[go.Bar(x=xg, y=yg, marker_color='salmon', name='girls'),
                      go.Bar(x=xb, y=yb, marker_color='skyblue', name='boys')
                     ]).update_xaxes(categoryorder='total descending')

fig.update_layout(title='<b>Out of School Children, Sub-Saharan Africa (2010 -2014) <b>',
                  xaxis_title='<b>Countries<b>',
                  yaxis_title='<b>Nr. Children<b>',
                  titlefont={'size': 20, 'family':'Serif'},
                  template='simple_white',
                  width=850, height=550,                  
                )

fig.show()

#### Observations:
- Generally more girls are out of school than boys which is in agreement with info from UNICEF.
- The data we have for the time periode we are considering is however spares (around 50% is NaN). Accurate comparision of gender disparity (in terms of children schooling) would improve with more data.

In [ ]:
girls = OutOfSchool.pivot(index = 'Year', columns = 'CountryName', values='Child_OutOfSchool_Prim_girls')
print('GIRLS')
display(girls.T.head(48).style.highlight_null('lightsalmon'))
boys = OutOfSchool.pivot(index = 'Year', columns = 'CountryName', values='Child_OutOfSchool_Prim_boys')
print('BOYS')
display(boys.T.head(48).style.highlight_null('skyblue'))

GIRLS


Year,2010,2011,2012,2013,2014
CountryName,,,,,
Angola,376262,455536,nan,nan,nan
Benin,nan,92509,nan,nan,nan
Botswana,nan,nan,12979,nan,nan
Burkina Faso,540522,484541,474376,466979,nan
Burundi,41263,nan,nan,77718,nan
Cabo Verde,1962,1769,1269,902,nan
Cameroon,nan,nan,198362,nan,nan
Central African Republic,133868,139288,127690,nan,nan
Chad,nan,469460,nan,275418,nan


BOYS


Year,2010,2011,2012,2013,2014
CountryName,,,,,
Angola,119896,57382,nan,nan,nan
Benin,nan,1089,nan,nan,nan
Botswana,nan,nan,15425,nan,nan
Burkina Faso,492409,443212,442668,445447,nan
Burundi,39887,nan,nan,326,nan
Cabo Verde,1383,938,354,179,nan
Cameroon,nan,nan,8234,nan,nan
Central African Republic,62292,72918,65962,nan,nan
Chad,nan,300981,nan,41009,nan


### 2.5 Child Labor

"Nearly 1 in 10 children are subjected to child labour worldwide, with some forced into hazardous work through trafficking." 

"Economic hardship exacts a toll on millions of families worldwide – and in some places, it comes at the price of a child’s safety. Roughly 160 million children were subjected to child labour at the beginning of 2020, with 9 million additional children at risk due to the impact of COVID-19.

This accounts for nearly 1 in 10 children worldwide. Almost half of them are in hazardous work that directly endangers their health and moral development.

Children may be driven into work for various reasons. Most often, child labour occurs when families face financial challenges or uncertainty – whether due to poverty, sudden illness of a caregiver, or job loss of a primary wage earner.

The consequences are staggering. Child labour can result in extreme bodily and mental harm, and even death. It can lead to slavery and sexual or economic exploitation. And in nearly every case, it cuts children off from schooling and health care, restricting their fundamental rights and threatening their futures.

Migrant and refugee children – many of whom have been uprooted by conflict, disaster or poverty – also risk being forced into work and even trafficked, especially if they are migrating alone or taking irregular routes with their families.

Trafficked children are often subjected to violence, abuse and other human rights violations. And some may be forced to break the law. For girls, the threat of sexual exploitation looms large, while boys may be exploited by armed forces or groups." [UNICEF (ref.6)](https://www.unicef.org/protection/child-labour)

> Let's now look for the right indicator code and see the data

> We are looking for IndicatorCode for `Children in employment, total (% of children ages 7-14)` , which is `SL.TLF.0714.ZS`

In [ ]:
indicator_type_labour = pd.read_sql("""
SELECT SeriesCode, IndicatorName, LongDefinition
FROM Series
WHERE Topic LIKE '%Social Protection & Labor: Economic activity%';
""", 
                             connect)
indicator_type_labour

,SeriesCode,IndicatorName,LongDefinition
0,SL.TLF.0714.SW.TM,"Average working hours of children, study and w...",Average working hours of children studying and...
1,SL.TLF.0714.SW.FE.TM,"Average working hours of children, study and w...",Average working hours of children studying and...
2,SL.TLF.0714.SW.MA.TM,"Average working hours of children, study and w...",Average working hours of children studying and...
3,SL.TLF.0714.WK.TM,"Average working hours of children, working onl...",Average working hours of children working only...
4,SL.TLF.0714.WK.FE.TM,"Average working hours of children, working onl...",Average working hours of children working only...
...,...,...,...
70,SL.EMP.VULN.MA.ZS,"Vulnerable employment, male (% of male employm...",Vulnerable employment is unpaid family workers...
71,SL.EMP.VULN.ZS,"Vulnerable employment, total (% of total emplo...",Vulnerable employment is unpaid family workers...
72,SL.EMP.WORK.FE.ZS,"Wage and salaried workers, female (% of female...",Wage and salaried workers (employees) are thos...
73,SL.EMP.WORK.ZS,"Wage and salaried workers, total (% of total e...",Wage and salaried workers (employees) are thos...


In [ ]:
for text in enumerate(indicator_type_labour['IndicatorName'].tolist()):
    print(text)
    print('')

(0, 'Average working hours of children, study and work, ages 7-14 (hours per week)')

(1, 'Average working hours of children, study and work, female, ages 7-14 (hours per week)')

(2, 'Average working hours of children, study and work, male, ages 7-14 (hours per week)')

(3, 'Average working hours of children, working only, ages 7-14 (hours per week)')

(4, 'Average working hours of children, working only, female, ages 7-14 (hours per week)')

(5, 'Average working hours of children, working only, male, ages 7-14 (hours per week)')

(6, 'Child employment in agriculture (% of economically active children ages 7-14)')

(7, 'Child employment in agriculture, female (% of female economically active children ages 7-14)')

(8, 'Child employment in agriculture, male (% of male economically active children ages 7-14)')

(9, 'Child employment in manufacturing (% of economically active children ages 7-14)')

(10, 'Child employment in manufacturing, female (% of female economically active children 

In [ ]:
df_SSA_labor = pd.read_sql("""

SELECT Year, Value AS Child_empl, CountryName
FROM Indicators
WHERE CountryName IN 

('Angola',
 'Benin','Botswana','Burkina Faso','Burundi','Cabo Verde','Cameroon',
 'Central African Republic','Chad','Comoros','Congo',"Côte d'Ivoire",
 'Dem. Rep. Congo','Equatorial Guinea','Eritrea','Ethiopia','Gabon',
 'Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Madagascar',
 'Malawi','Mali','Mauritania','Mauritius','Mozambique','Namibia','Niger',
 'Nigeria','Rwanda','São Tomé and Principe','Senegal','Seychelles','Sierra Leone',
 'Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','The Gambia',
 'Togo','Uganda','Zambia','Zimbabwe') 
 
 AND  
 
 IndicatorCode = 'SL.TLF.0714.ZS' AND Year IN (2010, 2011, 2012, 2013, 2014)
 
 """, connect)

df_SSA_labor.head(3)


,Year,Child_empl,CountryName
0,2010,50.3,Burkina Faso
1,2010,31.9,Burundi
2,2010,37.2,Central African Republic


In [ ]:
CountriesWithNoData(df_SSA_labor)

Only 21 countries out of the 48 have data for our current query.

Countries with no data in our current query are: 

1 Angola
2 Botswana
3 Cabo Verde
4 Comoros
5 Congo
6 Côte d'Ivoire
7 Dem. Rep. Congo
8 Equatorial Guinea
9 Eritrea
10 Ghana
11 Guinea-Bissau
12 Kenya
13 Lesotho
14 Madagascar
15 Mauritania
16 Mauritius
17 Mozambique
18 Namibia
19 São Tomé and Principe
20 Seychelles
21 Somalia
22 South Africa
23 South Sudan
24 Sudan
25 The Gambia
26 Zambia
27 Zimbabwe


In [ ]:
df_labor = df_SSA_labor.groupby(["CountryName"])['Child_empl'].mean().sort_values(ascending=False).reset_index()


fig = go.Figure(data=[go.Bar(x=df_labor['CountryName'], 
                             y=df_labor['Child_empl'],
                            marker_color='crimson'),                      
                     ]
               )

fig.update_layout(title='<b>Children In Employment, Total (% of children ages 7-14)<b>',
                  xaxis_title='<b>Countries<b>',
                  yaxis_title='<b>Percent (%) of total<b>',
                  titlefont={'size': 20, 'family':'Serif'},
                  template='simple_white',
                  width=900, height=550,
                )
fig.show()

In [ ]:
fig = px.choropleth(df_SSA_labor, locations="CountryName", locationmode='country names',
                    color="Child_empl",
                    hover_name="CountryName",
                    fitbounds="locations",
                    title = 'Sub-Saharan Africa Child Employment:  2010 - 2014', 
                    width=750, height= 500,
                    template = 'plotly')

fig.update(layout=dict(titlefont={'size': 24, 'family': 'Serif'}))

fig.show()

#### Observations:
- We note that all countries do not appear on the list, only 21 out of the 48 counties have data for this query.
- Cameroon at 62% is the highest child labour exploiter. 

### End Note:

As stated from the outset, the main objective of this notebook was to practice SQL. To that end this notebook serves as a *first* (only basic queries were covered) learning step in the world of SQL and querying data bases. Along the way I have also tried to look at and visualize some of the socio-economic data of the Sub-Saharan Africa region for years 2010 - 2014. 

### References: 

[1]. https://www.kaggle.com/code/jonaspalucibarbosa/wdi-brazil-unemployment-using-sql-for-analysis

[2]. https://www.geeksforgeeks.org/how-to-list-tables-using-sqlite3-in-python/

[3]. https://stackoverflow.com/questions/44545921/in-pandas-how-to-concatenate-horizontally-and-then-remove-the-redundant-columns

[4]. https://www.investopedia.com/terms/g/gdp.

[5]. https://landportal.org/taxonomy/term/8165

[6]. https://www.unicef.org/protection/child-labour

[7]. https://www.worldatlas.com/articles/what-are-the-major-natural-resources-of-mauritius.html

[8]. https://mode.com/sql-tutorial/introduction-to-sql/

---
##### Check list (SQL Basics) according to ref. 8:

- SELECT
- LIMIT
- WHERE
- Comparison Operators
- Logical Operators
- LIKE
- IN
- BETWEEN
- IS NULL
- AND
- OR
- NOT
- ORDER BY

---